# ⚠ Warning

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gl/OpinionatedGeek%2Fmango-explorer/HEAD?filepath=Pandas.ipynb) _🏃‍♀️ To run this notebook press the ⏩ icon in the toolbar above._

# 🥭 Mango + Pandas 🐼🐼

This notebook loads margin account data into a Pandas `DataFrame`.

The `DataFrame` is then queried for the total assets and liabilities, the Top 10 margin accounts with the most assets and the most liabilities, and then the Top 10 margin accounts closest to liquidation.

The data remains in the `DataFrame` called `df` so you can easily add your own queries and analyses.

In [ ]:
import pandas as pd
import time

from Context import default_context
from Classes import Group, MarginAccount, OpenOrders


In [ ]:
start_time = time.time()

print("Loading group...")
group = Group.load(default_context)
print(f"Done. Time taken: {time.time() - start_time}")

print("Loading prices...")
prices = group.get_prices()
print(f"Done. Time taken: {time.time() - start_time}")

print("Loading margin accounts...")
margin_accounts = MarginAccount.load_all_for_group(default_context, default_context.program_id, group)
print(f"Done. Time taken: {time.time() - start_time}")

print("Loading open orders accounts...")
open_orders = OpenOrders.load_raw_open_orders_accounts(default_context, group)
print(f"Done. Time taken: {time.time() - start_time}")

print("Installing open orders accounts...")
open_orders_by_address = {key: value for key, value in [(str(address), open_orders_account) for address, open_orders_account in open_orders]}
for margin_account in margin_accounts:
    margin_account.install_open_orders_accounts(group, open_orders_by_address)
print(f"Done. Time taken: {time.time() - start_time}")

print("Loading pandas dataframe...")
data = []
df_index = []
for index, margin_account in enumerate(margin_accounts):
    balance_sheet = margin_account.get_balance_sheet_totals(group, prices)
    df_index += [str(margin_account.address)]
    data += [{"Owner": margin_account.owner, "Liabilities": balance_sheet.liabilities, "Assets": balance_sheet.assets, "Settled Assets": balance_sheet.settled_assets, "Unsettled Assets": balance_sheet.unsettled_assets, "Collateral Ratio": balance_sheet.collateral_ratio}]
df = pd.DataFrame(data, index=df_index)

print(f"Done. Time taken: {time.time() - start_time}")

def render_styled(df: pd.DataFrame):
    return df.style.format({
    "Liabilities": "${:,.2f}",
    "Assets": "${:,.2f}",
    "Settled Assets": "${:,.2f}",
    "Unsettled Assets": "${:,.2f}",
    "Collateral Ratio": "{:,.2%}"
})


# Total assets and liabilities

In [ ]:
print(f"""
Total Assets:      ${df['Assets'].sum():>15,.2f}
Total Liabilities: ${df['Liabilities'].sum():>15,.2f}
Empty Accounts:     {len(df[df["Collateral Ratio"] == 0]):>15,}
Liquidatable:       {len(df[(df["Collateral Ratio"] != 0) & (df["Collateral Ratio"] <= 1.1)]):>15,}
🥭 Ripe Mangoes:    {len(df[(df["Collateral Ratio"] > 1.1) & (df["Collateral Ratio"] < 1.2)]):>15,}
""")

# Top 10 margin accounts with most assets

In [ ]:
render_styled(df.sort_values("Assets", ascending=False).head(10))

# Top 10 margin accounts with most liabilities

In [ ]:
render_styled(df.sort_values("Liabilities", ascending=False).head(10))

# Top 10 least collateralised margin accounts

Collect all margin accounts that have a non-zero Collateral Ratio (so have some liabilities). Then sort them from least-collateralised to most-collateralised.

In [ ]:
render_styled(df[df["Collateral Ratio"] != 0].sort_values("Collateral Ratio", ascending=True).head(10))